## _Reco. Track Evaluation_

- evaluate track reconstruction of GNN
- we have reconstructed tracks from _`trkx_from_gnn.py`_ (see its code breakdown in _`trkx_from_gnn.ipynb`_)


This is code breakdown of _`eval_reco_trkx.py`_ by using the similar script from _`gnn4itk/scripts/eval_reco_trkx.py`_

In [1]:
import glob, os, sys, yaml

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import torch
import time

In [4]:
from sklearn.cluster import DBSCAN
from multiprocessing import Pool
from functools import partial

In [5]:
# select a device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
sys.path.append('..')

In [7]:
from LightningModules.Processing import SttTorchDataReader

### _(1) Tracks from GNN_

* from _`tracks_from_gnn.py`_

In [8]:
raw_inputdir="../run_all/gnn_processed/pred"  # output of GNN stage as in test/pred
rec_inputdir="../run_all/gnn_segmenting/seg"  # output of trkx_from_gnn.sh
outputdir="../run_all/gnn_segmenting/eval"    # output of eval_reco_trkx.sh

In [9]:
# reco_track_path = "run/trkx_from_gnn"
reco_trkx_reader = SttTorchDataReader(rec_inputdir)

In [10]:
# what are the events?
reco_trkx_reader.all_evtids[:10]

['110000',
 '110001',
 '110002',
 '110003',
 '110004',
 '110005',
 '110006',
 '110007',
 '110008',
 '110009']

In [11]:
# fetch a single event
reco_trkx_data = reco_trkx_reader(110000)

In [12]:
reco_trkx_data.head()

,hit_id,track_id
0,52,0
1,77,1
2,127,2
3,178,3
4,1,4


In [13]:
# filter missed hits
reco_trkx_data.query("track_id==-1").head()

,hit_id,track_id
69,93,-1
79,145,-1
80,95,-1
85,96,-1
91,147,-1


In [14]:
# number of reco tracks
np.unique(reco_trkx_data.track_id.values)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)

In [15]:
# renaming
reconstructed = reco_trkx_data

### _(2) Track Evaluation_

- _Fixing `eval_reco_trkx.py`_

In [16]:
# arguments for script: args = parser.parse_args()
max_evts = 100
force = True
num_workers = 8
outdir = os.path.dirname(os.path.abspath(outputdir))
os.makedirs(outdir, exist_ok=True)

* Read raw CSV files to get truth information
* But I have torch-geometric data from the GNN stage

In [17]:
# fetch `raw` data
raw_trkx_reader = SttTorchDataReader(raw_inputdir)

In [18]:
n_tot_files = reco_trkx_reader.nevts
all_evtids = reco_trkx_reader.all_evtids
max_evts = max_evts if max_evts > 0 and max_evts <= n_tot_files else n_tot_files

In [19]:
raw_trkx_reader.all_evtids[:10]

['110000',
 '110001',
 '110002',
 '110003',
 '110004',
 '110005',
 '110006',
 '110007',
 '110008',
 '110009']

In [20]:
raw_trkx_data = raw_trkx_reader(110000)

In [21]:
# particles: ['particle_id', 'pt', 'eta', 'radius', 'vz'] where radius = sqrt(vx**2 + vy**2) and and ['vx', 'vy', 'vz'] are the production vertex

In [22]:
# raw_trkx_data
# raw_trkx_data.hid.numpy()
# raw_trkx_data.pid.int().numpy()

In [23]:
raw_trkx_data

Data(x=[129, 3], pid=[129], layers=[129], event_file='/home/adeak977/current/3_deeptrkx/ctd2022/train_all1/event0000110000', hid=[129], pt=[129], modulewise_true_edges=[2, 121], layerwise_true_edges=[2, 130], edge_index=[2, 475], y_pid=[475], scores=[950])

In [24]:
# reco:  ['hit_id', 'track_id']
reco_trkx_data.head()

,hit_id,track_id
0,52,0
1,77,1
2,127,2
3,178,3
4,1,4


In [25]:
# truth:  ['hit_id', 'particle_id']
truth = pd.DataFrame({'hit_id': raw_trkx_data.hid.numpy(), 'particle_id': raw_trkx_data.pid.int().numpy()}, columns=['hit_id', 'particle_id'])
truth.head()

,hit_id,particle_id
0,52,6
1,77,5
2,127,3
3,178,1
4,1,10


In [26]:
np.unique(truth.particle_id.values)

array([ 1,  2,  3,  4,  5,  6,  8, 10], dtype=int32)

In [27]:
# particles: ['particle_id', 'pt', 'eta', 'radius', 'vz']
particles = pd.DataFrame({'particle_id': raw_trkx_data.pid.int().numpy(), 'pt': raw_trkx_data.pt.numpy()}, columns=['particle_id', 'pt'])

In [28]:
particles.drop_duplicates(subset=['particle_id']).shape

(8, 2)

In [29]:
np.unique(particles.particle_id.values)

array([ 1,  2,  3,  4,  5,  6,  8, 10], dtype=int32)

### _Current torch-geometric data that I have?_

```
Data(x=[158, 3], pid=[158], layers=[158], event_file='event0000000900', hid=[158], pt=[158], modulewise_true_edges=[2, 148], layerwise_true_edges=[2, 153], edge_index=[2, 946], y_pid=[946], scores=[1892])
```

### _What I have in my torch-geometric data after GNNBuilder?_

1. x,y coordinates
2. hit_id (hid)
3. particle_id (pid)
4. pt
5. scores, etc

### _What I don't have in my torch-geometric data after GNNBuilder?_

1. eta
2. radius
3. vz


Can get `eta, radius, vz` if one re-process an event directly from **CSV** (similar to **ACTSCSVReader**) and add these variable in addition to what I already have.

### _Dissect Function_
We have this function: `evaluate_reco_tracks(truth_data, reco_data, particles_data)`

In [30]:
reco_trkx_data = reco_trkx_reader(110000)

In [31]:
reco_df = reco_trkx_data

In [32]:
raw_trkx_data  = raw_trkx_reader(110000)

In [33]:
# create truth, particle dataframes from Torch Data
_truth = pd.DataFrame({'hit_id': raw_trkx_data.hid.numpy(), 'particle_id': raw_trkx_data.pid.int().numpy()}, columns=['hit_id', 'particle_id'])
_particles = pd.DataFrame({'particle_id': raw_trkx_data.pid.int().numpy(), 'pt': raw_trkx_data.pt.numpy()},  columns=['particle_id', 'pt']).drop_duplicates(subset=['particle_id'])

In [34]:
truth_df = _truth
particles_df = _particles

In [35]:
truth_df.head()

,hit_id,particle_id
0,52,6
1,77,5
2,127,3
3,178,1
4,1,10


In [36]:
particles_df.head()

,particle_id,pt
0,6,0.294881
1,5,0.810588
2,3,1.041306
3,1,0.377064
4,10,0.680212


In [37]:
reco_df.head()

,hit_id,track_id
0,52,0
1,77,1
2,127,2
3,178,3
4,1,4


In [38]:
min_hits_truth=7
min_hits_reco=5
min_pt=0.
frac_reco_matched=0.5
frac_truth_matched=0.5

In [39]:
 # just in case particle_id == 0 included in truth.
if 'particle_id' in truth_df.columns:
    truth_df = truth_df[truth_df.particle_id > 0]

In [40]:
# get number of spacepoints in each reconstructed tracks
n_reco_hits = reco_df.track_id.value_counts(sort=False)\
    .reset_index().rename(
        columns={"index":"track_id", "track_id": "n_reco_hits"})

In [41]:
n_reco_hits.head(11)

,track_id,n_reco_hits
0,0,17
1,1,8
2,2,8
3,3,8
4,4,17
5,5,16
6,6,17
7,7,18
8,-1,8
9,8,3


In [42]:
# only tracks with a minimum number of spacepoints are considered
n_reco_hits = n_reco_hits[n_reco_hits.n_reco_hits >= min_hits_reco]
reco_df = reco_df[reco_df.track_id.isin(n_reco_hits.track_id.values)]

In [43]:
reco_df.describe()

,hit_id,track_id
count,117.000000,117.000000
mean,91.735043,3.555556
std,56.227585,2.637411
min,1.000000,-1.000000
25%,46.000000,1.000000
50%,83.000000,4.000000
75%,134.000000,6.000000
max,185.000000,7.000000


In [44]:
particles_df.describe()

,particle_id,pt
count,8.000000,8.000000
mean,4.875000,0.759993
std,3.044316,0.417517
min,1.000000,0.294881
25%,2.750000,0.358012
50%,4.500000,0.745400
75%,6.500000,1.095262
max,10.000000,1.317902


In [45]:
# get number of spacepoints in each particle
hits = truth_df.merge(particles_df, on='particle_id', how='left')
n_true_hits = hits.particle_id.value_counts(sort=False).reset_index().rename(
    columns={"index":"particle_id", "particle_id": "n_true_hits"})

In [46]:
hits.describe()

,hit_id,particle_id,pt
count,129.000000,129.000000,129.000000
mean,94.643411,5.162791,0.781398
std,54.806734,2.752245,0.388774
min,1.000000,1.000000,0.294881
25%,49.000000,3.000000,0.300856
50%,97.000000,5.000000,0.810588
75%,145.000000,8.000000,1.257133
max,185.000000,10.000000,1.317902


In [47]:
n_true_hits.describe()

,particle_id,n_true_hits
count,8.000000,8.000000
mean,4.875000,16.125000
std,3.044316,3.356763
min,1.000000,8.000000
25%,2.750000,16.750000
50%,4.500000,17.000000
75%,6.500000,18.000000
max,10.000000,18.000000


In [48]:
# only particles leaves at least min_hits_truth spacepoints 
# and with pT >= min_pt are considered.
particles_df = particles_df.merge(n_true_hits, on=['particle_id'], how='left')

In [49]:
particles_df.head(10)

,particle_id,pt,n_true_hits
0,6,0.294881,17
1,5,0.810588,18
2,3,1.041306,18
3,1,0.377064,8
4,10,0.680212,17
5,4,1.317902,16
6,2,1.257133,17
7,8,0.300856,18


In [50]:
particles_df.describe()

,particle_id,pt,n_true_hits
count,8.000000,8.000000,8.000000
mean,4.875000,0.759993,16.125000
std,3.044316,0.417517,3.356763
min,1.000000,0.294881,8.000000
25%,2.750000,0.358012,16.750000
50%,4.500000,0.745400,17.000000
75%,6.500000,1.095262,18.000000
max,10.000000,1.317902,18.000000


In [51]:
# filter particle that have pt > 1.
particles_df[particles_df.pt > 1].describe()

,particle_id,pt,n_true_hits
count,3.0,3.000000,3.0
mean,3.0,1.205447,17.0
std,1.0,0.145362,1.0
min,2.0,1.041306,16.0
25%,2.5,1.149219,16.5
50%,3.0,1.257133,17.0
75%,3.5,1.287518,17.5
max,4.0,1.317902,18.0


In [52]:
# filter particle that have pt < 1.
particles_df[particles_df.pt < 1].describe()

,particle_id,pt,n_true_hits
count,5.000000,5.000000,5.00000
mean,6.000000,0.492720,15.60000
std,3.391165,0.237446,4.27785
min,1.000000,0.294881,8.00000
25%,5.000000,0.300856,17.00000
50%,6.000000,0.377064,17.00000
75%,8.000000,0.680212,18.00000
max,10.000000,0.810588,18.00000


In [53]:
particles_df.head(10)

,particle_id,pt,n_true_hits
0,6,0.294881,17
1,5,0.810588,18
2,3,1.041306,18
3,1,0.377064,8
4,10,0.680212,17
5,4,1.317902,16
6,2,1.257133,17
7,8,0.300856,18


In [54]:
# filter particles in a range min=0.3 to max=0.8
minval = 0.
maxval = 0.9
particles_df[(particles_df.pt > minval) & (particles_df.pt < maxval)].head()

,particle_id,pt,n_true_hits
0,6,0.294881,17
1,5,0.810588,18
3,1,0.377064,8
4,10,0.680212,17
7,8,0.300856,18


In [55]:
is_trackable = particles_df.n_true_hits >= min_hits_truth

In [56]:
# event has 3 columnes [track_id, particle_id, hit_id]
event = pd.merge(reconstructed, truth, on=['hit_id'], how='left')

In [57]:
event.head()

,hit_id,track_id,particle_id
0,52,0,6
1,77,1,5
2,127,2,3
3,178,3,1
4,1,4,10


In [58]:
# n_common_hits and n_shared should be exactly the same 
# for a specific track id and particle id

In [59]:
# Each track_id will be assigned to multiple particles.
# To determine which particle the track candidate is matched to, 
# we use the particle id that yields a maximum value of n_common_hits / n_reco_hits,
# which means the majority of the spacepoints associated with the reconstructed
# track candidate comes from that true track.
# However, the other way may not be true.

In [60]:
reco_matching = event.groupby(['track_id', 'particle_id']).size()\
        .reset_index().rename(columns={0:"n_common_hits"})

In [61]:
reco_matching.head(15)

,track_id,particle_id,n_common_hits
0,-1,3,4
1,-1,5,4
2,0,6,17
3,1,5,8
4,2,3,8
5,3,1,8
6,4,10,17
7,5,4,16
8,6,2,17
9,7,8,18


In [62]:
# Each particle will be assigned to multiple reconstructed tracks
truth_matching = event.groupby(['particle_id', 'track_id']).size()\
    .reset_index().rename(columns={0:"n_shared"})

In [63]:
truth_matching.head(15)

,particle_id,track_id,n_shared
0,1,3,8
1,2,6,17
2,3,-1,4
3,3,2,8
4,3,8,2
5,3,9,1
6,3,10,3
7,4,5,16
8,5,-1,4
9,5,1,8


In [64]:
# add number of hits to each of the maching dataframe
reco_matching = reco_matching.merge(n_reco_hits, on=['track_id'], how='left')
truth_matching = truth_matching.merge(n_true_hits, on=['particle_id'], how='left')

# calculate matching fraction
reco_matching = reco_matching.assign(
    purity_reco=np.true_divide(reco_matching.n_common_hits, reco_matching.n_reco_hits))
truth_matching = truth_matching.assign(
    purity_true = np.true_divide(truth_matching.n_shared, truth_matching.n_true_hits))

In [65]:
# select the best match
reco_matching['purity_reco_max'] = reco_matching.groupby(
    "track_id")['purity_reco'].transform(max)
truth_matching['purity_true_max'] = truth_matching.groupby(
    "track_id")['purity_true'].transform(max)